In [27]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.optimizers import SGD, RMSprop
import logging
import yaml
from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import sys

In [31]:
settings = {'last_activation':'sigmoid', 'output_size': 17, 'loss':'binary_crossentropy', 'input_shape':(150,150), 
           'valid_size': 0.15, 'random_state':42}
inception_v3_phase_1 = {'name':'inception_v3', 'phase':'top_layer', 'batch_size': 16, 'optimiser':'rmsprop'}
inception_v3_phase_2 = {'name':'inception_v3', 'phase':'full_1', 'batch_size': 32, 'optimiser':'sgd',
                   'lr':0.01, 'momentum':0.9, 'train_from': 'inception_v3_top_layer_00_0.191.hdf5', 'epochs':100}

In [10]:
# create data generators
def get_labels():
    with open('../label_maps.yml', 'r') as lablels:
        label_data = yaml.load(lablels)
    inv_label_map = label_data['inv_label_map']
    label_map = label_data['label_map']
    return label_map, inv_label_map

label_map, inv_label_map = get_labels()

In [11]:
def load_images_and_resize(label_map, input_shape, valid_size, random_state, verbose=1):
    df_train = pd.read_csv('../data/train_v2.csv')
    if verbose:
        print('Loading images and resize to {}'.format(input_shape))
    x_train = []
    y_train = []
    for f, tags in tqdm(df_train.values, miniters=1000):
        img = cv2.imread('../data/train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 
        x_train.append(cv2.resize(img, input_shape))
        y_train.append(targets)
    y_train = np.array(y_train, np.uint8)
    x_train = np.array(x_train, np.uint8)
    
    x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                          y_train,
                                                          test_size=valid_size,
                                                          random_state=random_state)
    
    if verbose == 1:
        print('Train data size: {:.2f} GB'.format(sys.getsizeof(x_train)/1e9))
        print('input shape: {}'.format(x_train.shape))
        print('Train data size: {:.2f} GB'.format(sys.getsizeof(x_valid)/1e9))
        print('input shape: {}'.format(x_valid.shape))
    
    return x_train, x_valid, y_train, y_valid

x_train, x_valid, y_train, y_valid = load_images_and_resize(label_map, settings['input_shape'],
                                                            settings['valid_size'],
                                                            settings['random_state'] )

  0%|          | 0/40479 [00:00<?, ?it/s]

Loading images and resize to (150, 150)


100%|██████████| 40479/40479 [01:50<00:00, 366.66it/s]


Train data size: 2.32 GB
input shape: (34407, 150, 150, 3)
Train data size: 0.41 GB
input shape: (6072, 150, 150, 3)


In [12]:
train_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_generator = ImageDataGenerator(rescale=1. / 255)

In [23]:
def add_top_layer(base_model, settings):
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(settings['output_size'], activation=settings['last_activation'])(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False
        
    return model

In [ ]:
def train_cnn(x_train, y_train, x_valid, y_valid, settings, phase):
    file_path = '../models/{}_{}_'.format(phase['name'], phase['phase']) + '{epoch:02d}_{val_loss:.3f}.hdf5'
    model_checkpoint = ModelCheckpoint(file_path, monitor='val_loss',
                                   verbose=0, save_best_only=False, 
                                save_weights_only=False, mode='auto',
                                   period=1)
    es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto')
    
    # create the base pre-trained model
    if phase['phase'] == 'top_layer':
        if phase['name'] == 'inception_v3':
            print('Loading model {} with no top layer'.format(phase['name']))
            base_model = InceptionV3(weights='imagenet', include_top=False)
            model = add_top_layer(base_model, settings)
    elif 'full' in phase['phase']:
        if phase['name'] == 'inception_v3':
            # load model
            print('loading pre trained model..')
            model = load_model('../models/{}'.format(phase['train_from']))
            for layer in model.layers[:172]:
               layer.trainable = True
            for layer in model.layers[172:]:
               layer.trainable = True
    
    print('setting optimiser')
    if phase['optimiser'] == 'rmsprop':
        optimiser = RMSprop()
    if phase['optimiser'] == 'sgd':
        optimiser = SGD(lr=phase['lr'], momentum=phase['momentum'])
    
    # compile the model (should be done *after* setting layers to non-trainable)
    print('compiling model')
    model.compile(optimizer=optimiser, loss=settings['loss'], metrics=['accuracy'])
    
    print('Load generators')
    train_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
    train_generator_loaded = train_generator.flow(x_train, y_train, batch_size=phase['batch_size'])

    valid_generator = ImageDataGenerator(rescale=1. / 255)
    valid_generator_loaded = valid_generator.flow(x_valid, y_valid, batch_size=phase['batch_size'], 
                                                  shuffle=False)
    
    
    print('Starting training..')
    model.fit_generator(train_generator_loaded,
                    steps_per_epoch= len(x_train) / phase['batch_size'],
                    epochs=settings['epochs'], validation_data=valid_generator_loaded,
                    validation_steps = len(x_valid) / phase['batch_size'],
                    callbacks = [model_checkpoint, es], verbose=1)
    
    return model
    
model = train_cnn(x_train, y_train, x_valid, y_valid, settings, inception_v3_phase_2)

loading pre trained model..
setting optimiser
compiling model
Load generators
Starting training..
Epoch 1/20
 158/1075 [===>..........................] - ETA: 495s - loss: 0.1633 - acc: 0.9403